In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
from scipy import spatial, stats
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation

In [ ]:
def sample_clustered_points(points):
    from random import randrange
    """
    Given a set of points, returns a second set of points that are spatially
    related
    """
    new_points = [[z+randrange(0,11)-5, y+randrange(0,11)-5, x+randrange(0,11)-5] for z, y, x in points]
    return np.array(new_points)


In [ ]:

### Visualize Multivariate Points Within Sphere
scale = 35

random_set1 = stats.uniform.rvs(loc=0, scale=100, size=(100,3))
random_set2 = stats.uniform.rvs(loc=0, scale=100, size=(100,3))
# random_NG2 = sample_clustered_points(random_DTC)
# random_NG2 = stats.skewnorm.rvs(a=4, loc=50, scale=25, size=(100,3))

# DTC_tree = spatial.cKDTree(random_DTC)
NG2_tree = spatial.cKDTree(random_NG2)

# DTC_idx_in = DTC_tree.query_ball_point((50,50,50), scale)
NG2_idx_in = NG2_tree.query_ball_point((50,50,50), scale)

In [ ]:

# DTC_idx_out = np.ones(len(random_DTC), bool); DTC_idx_out[DTC_idx_in] = 0
NG2_idx_out = np.ones(len(random_NG2), bool); NG2_idx_out[NG2_idx_in] = 0

# DTC_inside, DTC_outside = random_DTC[DTC_idx_in], random_DTC[DTC_idx_out]
NG2_inside, NG2_outside = random_NG2[NG2_idx_in], random_NG2[NG2_idx_out]

NG2_ix, NG2_ox = NG2_inside[:, 0], NG2_outside[:, 0]
NG2_iy, NG2_oy = NG2_inside[:, 1], NG2_outside[:, 1]
NG2_iz, NG2_oz = NG2_inside[:, 2], NG2_outside[:, 2]

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
set_aspect_ratio_equal(ax)
ax.scatter(NG2_ix,NG2_iy,NG2_iz, color="#f00",marker="o", depthshade=True)
ax.scatter(NG2_ox,NG2_oy,NG2_oz,color="#ffaaaa",marker="o", depthshade=True)


# draw sphere
u, v = np.mgrid[0:2*np.pi:40j, 0:np.pi:40j]
cx = scale*np.cos(u)*np.sin(v)+50
cy = scale*np.sin(u)*np.sin(v)+50
cz = scale*np.cos(v)+50
ax.scatter(cx, cy, cz, color="gray", s=1, alpha=0.3)

In [ ]:

def generate_3DRipley_animation():
    # Set up the figure and subplots
    fig = plt.figure(figsize=(14, 6))
    ax1 = fig.add_subplot(121, projection='3d')
    set_aspect_ratio_equal(ax1)
    ax2 = fig.add_subplot(122)

    # Initial settings for the 3D scatter plot
    ax1.set_xlim(0, 100)
    ax1.set_ylim(0, 100)
    ax1.set_zlim(0, 100)
    ax1.set_title('3D Scatter Plot')
    ax1.set_xlabel('X axis')
    ax1.set_ylabel('Y axis')
    ax1.set_zlabel('Z axis')
    # ax1.view_init(azim=5)

    # Example of adding a semi-transparent plane at z=0
    # x_plane, y_plane = np.meshgrid(np.linspace(0, 100, 2), np.linspace(0, 100, 2))
    # z_plane = np.zeros_like(x_plane)
    # ax.plot_surface(x_plane, y_plane, z_plane, color='gray', alpha=0.9)

    # Plot a point at (50, 50, 50) with size 1 initially
    point, = ax1.plot([50], [50], [50], 'o', markersize=1, color='blue')

    # Initial settings for the 2D line plot
    ax2.set_xlim(0, max_radius+2)
    ax2.set_ylim(0, max(vol_equation))
    ax2.set_title('Volume of a Sphere as a Function of Radius')
    ax2.set_xlabel('Radius (r)')
    ax2.set_ylabel('Volume')
    line, = ax2.plot([], [], color='green')
    ax2.grid(True)

    # Generate random points
    random_NG2 = stats.uniform.rvs(loc=0, scale=100, size=(100, 3))
    NG2_tree = spatial.cKDTree(random_NG2)

    # Function to draw the wireframe sphere
    def draw_sphere(ax, scale):
        u, v = np.mgrid[0:2*np.pi:110j, 0:np.pi:110j]
        cx = scale*np.cos(u)*np.sin(v) + 50
        cy = scale*np.sin(u)*np.sin(v) + 50
        cz = scale*np.cos(v) + 50
        # ax.plot_wireframe(cx, cy, cz, color="green", alpha=0.3)\

        cx_flat, cy_flat, cz_flat = cx.flatten(), cy.flatten(), cz.flatten()

        # Determine which points are inside the 100x100x100 volume
        inside_condition = (cx_flat >= 0) & (cx_flat <= 100) & \
                           (cy_flat >= 0) & (cy_flat <= 100) & \
                           (cz_flat >= 0) & (cz_flat <= 100)

        colors = np.where(inside_condition, "gray", "red")
        # alpha = np.where(inside_condition, 0.3, 0.1)
        ax.scatter(cx, cy, cz, color=colors, s=1, alpha=0.3)

    # Initialize the line data
    line_data = ([], [])

    # Update function for the animation
    def update(frame):
        ax1.clear()
        ax1.view_init(azim=frame/10 + 45, elev=10)
        ax1.set_xlim(0, 100)
        ax1.set_ylim(0, 100)
        ax1.set_zlim(0, 100)
        ax1.set_xlabel('X axis')
        ax1.set_ylabel('Y axis')
        ax1.set_zlabel('Z axis')
        ax1.set_title('3D Wireframe Sphere')

        # Update the wireframe sphere with the new scale
        # scaling_factor = max_radius / len(r)
        draw_sphere(ax1, frame / 5 + 1)  # Scale grows over time

        # Determine inside and outside points
        NG2_idx_in = NG2_tree.query_ball_point((50, 50, 50), frame / 5)
        NG2_idx_out = np.ones(len(random_NG2), bool)
        NG2_idx_out[NG2_idx_in] = False
        NG2_inside, NG2_outside = random_NG2[NG2_idx_in], random_NG2[NG2_idx_out]

        # Plot points
        ax1.scatter(NG2_inside[:, 0], NG2_inside[:, 1], NG2_inside[:, 2], color="#f00", marker="o", depthshade=True)
        ax1.scatter(NG2_outside[:, 0], NG2_outside[:, 1], NG2_outside[:, 2], color="#ffaaaa", marker="o", depthshade=True)

        # Update the 2D line plot on the right to reveal the line up to the current frame
        if frame < len(r):
            line_data[0].append(r[int(frame)])
            line_data[1].append(vol_equation[int(frame)])
            line.set_data(line_data[0], line_data[1])

        return ax1, ax2

    # Creating the animation with the updated logic
    ani = FuncAnimation(fig, update, frames=len(r), blit=False, interval=50)

    plt.close()  # Prevent displaying a static plot in the notebook

    # ani  # Display the animation
    ani.save('animation.mp4', writer='ffmpeg')

# generate_3DRipley_animation()

In [ ]:

## Visualize Univariate Points Within Sphere

test_radius = 60
test_center = (50, 50, 50)

test_random_points = stats.uniform.rvs(0, 100, (100,3))
test_tree = spatial.cKDTree(test_random_points)
idx_in = test_tree.query_ball_point(test_center, test_radius)

idx_out = np.ones(len(test_random_points), bool)
idx_out[idx_in] = 0

pts_inside_radius = test_random_points[idx_in]
pts_outside_radius = test_random_points[idx_out]

print(len(test_random_points), len(pts_inside_radius), len(pts_outside_radius))

ix, ox = pts_inside_radius[:, 0], pts_outside_radius[:, 0]
iy, oy = pts_inside_radius[:, 1], pts_outside_radius[:, 1]
iz, oz = pts_inside_radius[:, 2], pts_outside_radius[:, 2]

fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(projection='3d')
set_aspect_ratio_equal(ax)
ax.scatter(ix,iy,iz, color="b",marker="o", depthshade=0)
ax.scatter(ox,oy,oz,color="r",marker="o", depthshade=1)

# Remove the grid
ax.grid(False)
# Optionally, remove the background pane as well
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
# Optionally, also remove the pane lines
ax.xaxis.pane.set_edgecolor('w')
ax.yaxis.pane.set_edgecolor('w')
ax.zaxis.pane.set_edgecolor('gray')

# draw sphere
u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
cx = test_radius*np.cos(u)*np.sin(v)+test_center[0]
cy = test_radius*np.sin(u)*np.sin(v)+test_center[1]
cz = test_radius*np.cos(v)+test_center[2]
ax.plot_wireframe(cx, cy, cz, color="gray", alpha=0.3)

ax.view_init(elev=30, azim=-135)
# plt.savefig("/home/dkermany/Desktop/sphere_large_1.png", dpi=600)

In [ ]:
# Run Multivariate (Cross) Ripley on Random Points data
radii = np.arange(2, 67) 
rand_rstats = monte_carlo(random_set1, mask, radii, random_set2, n_samples=100, n_processes=55, boundary_correction=False)
results = run_ripley(random_set1, random_set2, mask, radii, n_processes=55, boundary_correction=False)
results_w = run_ripley(random_set1, random_set2, mask, radii, n_processes=55, boundary_correction=True)


# Plot Random Points Results
plt.figure(figsize=(6,6))

# Unweighted K function
sns.lineplot(data=results, x="Radius (r)", y="K(r)", alpha=0.8, color=sns.color_palette()[0], label="unweighted")

# Weighted K function
sns.lineplot(data=results_w, x="Radius (r)", y="K(r)", alpha=0.8, color=sns.color_palette()[1], label="weighted")

max_radius = 68

# Expected K function
r = np.linspace(-1, max_radius, 500)
vol_equation = (4/3) * np.pi * r**3
expected_df = pd.DataFrame({
    "radius": r,
    "expected": vol_equation,
})
sns.lineplot(data=expected_df, x="radius", y="expected", linewidth=2, linestyle="dotted", color="#aaa", label="expected")
plt.text(max_radius-3, 4/3 * np.pi * (max_radius-1)**3 - 10000, r'y = $\frac{4}{3}\pi r^3$', fontsize=12, horizontalalignment='right', color="#aaa")

plt.show()